# 附录 10.1：提示词链接

- [课程内容](#lesson)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# 已重写以接受任意长度的消息列表
def get_completion(messages, system_prompt=""):
    # 构建消息列表
    formatted_messages = []
    if system_prompt:
        formatted_messages.append({"role": "system", "content": system_prompt})
    
    formatted_messages.extend(messages)
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=formatted_messages
    )
    return response.choices[0].message.content

---

## 课程内容

俗话说，"写作就是重写"。事实证明，**当被要求这样做时，模型通常可以提高其响应的准确性**！

有很多方法可以提示模型"重新思考"。那些感觉自然地要求人类重新检查其工作的方式通常也适用于模型。（查看我们的[提示词链接文档](https://docs.anthropic.com/claude/docs/chain-prompts)以获取何时以及如何使用提示词链接的更多示例。）

### 示例

在这个示例中，我们要求模型想出十个单词...但其中一个或多个不是真实的单词。

In [ ]:
# 初始提示词
first_user = "列出十个都以字母'ab'结尾的单词。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print(first_response)

**要求模型使其答案更准确**修复了错误！

下面，我们拉取了上面模型的错误响应，并在对话中添加了另一轮，要求模型修复其之前的答案。

In [ ]:
second_user = """请为所有不是真实单词的'单词'找到替换。"""

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

但是模型是否只是因为我们告诉它才修改答案？如果我们一开始就有了正确答案会怎样？模型会失去信心吗？在这里，我们在 `first_response` 的位置放置了一个正确的响应，并要求它再次进行双重检查。

In [ ]:
first_user = "列出十个都以字母'ab'结尾的单词。"

first_response = """以下是10个以字母'ab'结尾的单词：

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "请为所有不是真实单词的'单词'找到替换。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

您可能会注意到，如果您从上面的代码块生成几次响应，模型大部分时间会保持单词不变，但偶尔仍会更改单词，即使它们都已经是正确的。我们能做些什么来缓解这个问题？根据第8章，我们可以给模型一个出路！让我们再试一次。

In [ ]:
first_user = "列出十个都以字母'ab'结尾的单词。"

first_response = """以下是10个以字母'ab'结尾的单词：

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "请为所有不是真实单词的'单词'找到替换。如果所有单词都是真实的单词，请返回原始列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

尝试从上面的代码生成几次响应，您会看到模型现在更善于坚持其立场。

您还可以使用提示词链接来**要求模型改善其响应**。下面，我们要求模型首先写一个故事，然后改进它写的故事。您的个人喜好可能会有所不同，但许多人可能会同意模型的第二个版本更好。

首先，让我们生成模型的第一个版本故事。

In [ ]:
# 初始提示词
first_user = "写一个关于喜欢跑步的女孩的三句话短故事。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print(first_response)

现在让我们让模型改进其第一稿。

In [ ]:
second_user = "让故事更好。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

这种替换形式非常强大。我们一直在使用替换占位符来传递列表、单词、模型的前期响应等等。您还可以**使用替换来做我们称为"函数调用"的事情，即要求模型执行某个函数，然后获取该函数的结果，并要求模型对结果做更多的处理**。它的工作方式与任何其他替换一样。更多信息请参见下一个附录。

下面是另一个例子，取模型的一次调用结果并将其插入到另一个更长的调用中。让我们从第一个提示词开始（这次包括预填充模型的响应）。

In [ ]:
first_user = """从下面的文本中找到所有姓名：

"嘿，杰西。是我，艾琳。我打电话是关于乔伊明天举办的聚会。凯莎说她会来，我想梅尔也会在那里。"""

prefill = "<names>"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(first_response)

让我们将这个姓名列表传递到另一个提示词中。

In [ ]:
second_user = "按字母顺序排列这个列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

现在您已经了解了提示词链接，请转到附录10.2了解如何使用提示词链接实现函数调用。

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示词示例的区域，您可以调整提示词来看看它如何影响模型的响应。

In [ ]:
# 初始提示词
first_user = "列出十个都以字母'ab'结尾的单词。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "请为所有不是真实单词的'单词'找到替换。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "列出十个都以字母'ab'结尾的单词。"

first_response = """以下是10个以字母'ab'结尾的单词：

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "请为所有不是真实单词的'单词'找到替换。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "列出十个都以字母'ab'结尾的单词。"

first_response = """以下是10个以字母'ab'结尾的单词：

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "请为所有不是真实单词的'单词'找到替换。如果所有单词都是真实的单词，请返回原始列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

In [ ]:
# 初始提示词
first_user = "写一个关于喜欢跑步的女孩的三句话短故事。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print(first_response)

In [ ]:
first_user = """从下面的文本中找到所有姓名：

"嘿，杰西。是我，艾琳。我打电话是关于乔伊明天举办的聚会。凯莎说她会来，我想梅尔也会在那里。\""""

prefill = "<names>"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# 存储并打印模型的响应
first_response = get_completion(messages)
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(first_response)


In [ ]:
second_user = "按字母顺序排列这个列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))


In [ ]:
second_user = "让故事更好。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

In [ ]:
second_user = "按字母顺序排列这个列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印模型的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(messages))

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))